In [1]:
import numpy as np
import tensorflow_probability as tfp
import pandas as pd
from bisect import bisect_left
from scipy import special
from scipy.special import erf
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from numpy.linalg import inv
import tensorflow as tf
import scipy.linalg
import scipy.spatial
import gpflow
from gpflow.ci_utils import ci_niter
from gpflow import set_trainable
from scipy.stats.distributions import chi2
import pandas as pd
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from gpflow.utilities import print_summary, positive
np.set_printoptions(suppress=True)
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import pymc3

from utilities import (K_sqexp_kernel,K_new_kernel, compute_prior_hyperparameters, fit_1d_model, predict_in_observations, fit_Hand, y_exp_Hand, y_exp, fit_3d_model, find_nearest, K_log, K_multiplicative)

f64 = gpflow.utilities.to_default_float

def trapezoidal_area(xyz):
    """Calculate volume under a surface defined by irregularly spaced points
    using delaunay triangulation. "x,y,z" is a <numpoints x 3> shaped ndarray."""
    d = scipy.spatial.Delaunay(xyz[:,:2])
    tri = xyz[d.vertices]

    a = tri[:,0,:2] - tri[:,1,:2]
    b = tri[:,0,:2] - tri[:,2,:2]
    proj_area = np.cross(a, b).sum(axis=-1)
    zavg = tri[:,:,2].sum(axis=1)
    vol = zavg * np.abs(proj_area) / 6.0
    return vol.sum()


WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


In [2]:
N=5000 # number splits in Hand model

df = pd.read_csv('GrecoSimulatedData.csv', sep=';')

X = np.linspace(0,50,10)[:, None]
#k = K_multiplicative()
k =  K_new_kernel()
K = k(X, X)


print(K)

df = df.sort_values(by=['Dose1','Dose2'])
Effect = df['Response'].values.reshape(-1,1).copy()
Dose_A = df['Dose1'].values.astype(float).copy()
Dose_B = df['Dose2'].values.astype(float).copy()
Dose_AB = np.concatenate((Dose_A.reshape(-1,1), Dose_B.reshape(-1,1)), axis=1)

Effect_B = df[df['Dose1'] == 0]['Response'].to_numpy().reshape(-1,1).astype(float)
Effect_A = df[df['Dose2'] == 0]['Response'].to_numpy().reshape(-1,1).astype(float)
Dose_A = df[df['Dose2']==0]['Dose1'].to_numpy().reshape(-1,1).astype(float)
Dose_B = df[df['Dose1']==0]['Dose2'].to_numpy().reshape(-1,1).astype(float)

# hyperparameters of the priors
A_max  = np.max(Dose_A)
B_max = np.max(Dose_B)

alphaA, betaA = compute_prior_hyperparameters(A_max/5, 0.1*A_max)
alphaB, betaB = compute_prior_hyperparameters(B_max/2, 0.1*B_max)

eff_max_a = np.max(Effect_A)
eff_max_b = np.max(Effect_B)
eff_max = np.max([eff_max_a, eff_max_b])

alpha_var, beta_var = compute_prior_hyperparameters(2.5*eff_max, 0.1*eff_max)

data = pd.concat([pd.DataFrame(Dose_AB), pd.DataFrame(Effect)], axis=1)
data.columns = ['Dose_A', 'Dose_B', 'Effect']

X1 = data[data['Dose_B']==0]['Dose_A'].to_numpy().reshape(-1,1).astype(float)
Y1 = data[data['Dose_B']==0]['Effect'].to_numpy().reshape(-1,1).astype(float)

X2 = data[data['Dose_A']==0]['Dose_B'].to_numpy().reshape(-1,1).astype(float)
Y2 = data[data['Dose_A']==0]['Effect'].to_numpy().reshape(-1,1).astype(float)

Dose_A = data[data['Dose_B']==0]['Dose_A'].to_numpy().reshape(-1,1).astype(float)
Effect_A = data[data['Dose_B']==0]['Effect'].to_numpy().reshape(-1,1).astype(float)

Dose_B = data[data['Dose_A']==0]['Dose_B'].to_numpy().reshape(-1,1).astype(float)
Effect_B = data[data['Dose_A']==0]['Effect'].to_numpy().reshape(-1,1).astype(float)



tf.Tensor(
[[   -1.4106894    -43.5831179   -150.5809199   -321.46376201
   -556.14822809  -854.61756754 -1216.86693578 -1642.89459508
  -2132.69982471 -2686.28229238]
 [  -43.5831179    -60.62481216  -148.74549431  -307.79876292
   -534.78737706  -828.02735728 -1186.58948312 -1609.93680986
  -2097.74380746 -2649.80462201]
 [ -150.5809199   -148.74549431  -206.91852723  -338.857839
   -543.51217019  -818.71231892 -1162.70032063 -1574.19704067
  -2052.28687903 -2596.3101845 ]
 [ -321.46376201  -307.79876292  -338.857839    -440.29183461
   -615.85532769  -864.83092504 -1185.69168013 -1576.96206107
  -2037.40136171 -2566.00979022]
 [ -556.14822809  -534.78737706  -543.51217019  -615.85532769
   -760.74473428  -979.88434171 -1272.77677404 -1638.30836895
  -2075.28326956 -2582.60033354]
 [ -854.61756754  -828.02735728  -818.71231892  -864.83092504
   -979.88434171 -1168.27722626 -1430.9755084  -1767.61790606
  -2177.36164222 -2659.23538962]
 [-1216.86693578 -1186.58948312 -1162.70032063 -1

In [3]:
init_lengthscale = 1.01
init_variance = 0.01
init_variance_lik = 1.0

m = gpflow.models.GPR(data=(X1, Y1), kernel=k,  mean_function=None)
m.kernel.lengthscale.assign(init_lengthscale)
m.kernel.variance.assign(init_variance)
m.likelihood.variance.assign(init_variance_lik)

opt = gpflow.optimizers.Scipy()
opt_logs = opt.minimize(m.training_loss,
                        m.trainable_variables,method='BFGS',
                        options=dict(maxiter=100))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'


InvalidArgumentError:  Cholesky decomposition was not successful. The input might not be valid.
	 [[node Cholesky (defined at /Users/Yuliya/opt/anaconda3/envs/gpflow2new/lib/python3.6/site-packages/gpflow/models/gpr.py:71) ]] [Op:__inference__tf_eval_769]

Errors may have originated from an input operation.
Input Source operations connected to node Cholesky:
 set_diag (defined at /Users/Yuliya/opt/anaconda3/envs/gpflow2new/lib/python3.6/site-packages/gpflow/models/gpr.py:70)

Function call stack:
_tf_eval


In [5]:
## generate test points for prediction
xx = np.linspace(-0.1, 20.1, 100).reshape(100, 1)  # test points must be of shape (N, D)

## predict mean and variance of latent GP at test points
mean, var = m.predict_f(xx)
print(mean)

## generate 10 samples from posterior
tf.random.set_seed(1)  # for reproducibility
samples = m.predict_f_samples(xx, 10)  # shape (10, 100, 1)

## plot
plt.figure(figsize=(12, 6))
plt.plot(X1, Y1, "kx", mew=2)
plt.plot(xx, mean, "C0", lw=2)
plt.fill_between(
    xx[:, 0],
    mean[:, 0] - 1.96 * np.sqrt(var[:, 0]),
    mean[:, 0] + 1.96 * np.sqrt(var[:, 0]),
    color="C0",
    alpha=0.2,
)

InvalidArgumentError: cannot compute Mul as input #1(zero-based) was expected to be a double tensor but is a float tensor [Op:Mul] name: mul/

In [8]:
k(xx,xx)[0,:]

<tf.Tensor: shape=(100,), dtype=float64, numpy=
array([   -63.47106118,    -78.77732339,    -98.62008176,   -122.94317844,
         -151.68311625,   -184.77869473,   -222.17449037,   -263.82173117,
         -309.67808925,   -359.70706402,   -413.87725384,   -472.16164462,
         -534.53696617,   -600.9831308 ,   -671.48275321,   -746.02074421,
         -824.58396858,   -907.16095767,   -993.7416679 ,  -1084.31727783,
        -1178.88001709,  -1277.42302204,  -1379.94021355,  -1486.42619327,
        -1596.87615532,  -1711.28581091,  -1829.65132373,  -1951.96925459,
        -2078.23651359,  -2208.45031891,  -2342.6081611 ,  -2480.70777204,
        -2622.74709796,  -2768.72427587,  -2918.63761291,  -3072.4855683 ,
        -3230.26673744,  -3391.97983785,  -3557.6236969 ,  -3727.19724077,
        -3900.69948482,  -4078.12952495,  -4259.48652995,  -4444.76973463,
        -4633.97843375,  -4827.11197654,  -5024.16976182,  -5225.1512336 ,
        -5430.05587712,  -5638.8832153 ,  -5851.6328